# K-Means Clustering

Distance metrics used:
- Euclidean
- Hamming
- Cosine
- Mahalanobis (measure of the distance between a point P and distribution D)

Dataset:
This project explores the relationship between Social Media, Salary, Influence, Performance and Team Valuation in the NBA

In [5]:
import pandas as pd
team_data = "https://raw.githubusercontent.com/noahgift/socialpowernba/master/data/nba_2017_att_val_elo_win_housing.csv"
val_housing_win_df = pd.read_csv(team_data)
numerical_df = val_housing_win_df.loc[:,["TOTAL_ATTENDANCE_MILLIONS", "ELO", "VALUE_MILLIONS", "MEDIAN_HOME_PRICE_COUNTY_MILLIONS"]]

In [7]:
val_housing_win_df.head()


,TEAM,GMS,PCT_ATTENDANCE,WINNING_SEASON,TOTAL_ATTENDANCE_MILLIONS,VALUE_MILLIONS,ELO,CONF,COUNTY,MEDIAN_HOME_PRICE_COUNTY_MILLIONS,COUNTY_POPULATION_MILLIONS
0,Chicago Bulls,41,104,1,0.888882,2500,1519,East,Cook,269900.0,5.20
1,Dallas Mavericks,41,103,0,0.811366,1450,1420,West,Dallas,314990.0,2.57
2,Sacramento Kings,41,101,0,0.721928,1075,1393,West,Sacremento,343950.0,1.51
3,Miami Heat,41,100,1,0.805400,1350,1569,East,Miami-Dade,389000.0,2.71
4,Toronto Raptors,41,100,1,0.813050,1125,1600,East,York-County,390000.0,1.10
